In [1]:
import pandas as pd

df = pd.read_csv("M1_data.csv")

df.head()


,trust_apple,interest_computers,age_computer,user_pcmac,appleproducts_count,familiarity_m1,f_batterylife,f_price,f_size,f_multitasking,...,f_neural,f_synergy,f_performanceloss,m1_consideration,m1_purchase,gender,age_group,income_group,status,domain
0,No,4,8,PC,0,No,5,4,3,4,...,2,1,1,1,Yes,Male,2,2,Student,Science
1,Yes,2,4,PC,1,No,5,5,5,3,...,2,2,4,2,No,Male,2,3,Employed,Finance
2,Yes,5,6,PC,0,No,3,4,2,4,...,2,2,2,4,Yes,Male,2,2,Student,IT & Technology
3,Yes,2,6,Apple,4,No,4,3,3,4,...,4,4,3,2,No,Female,2,2,Student,Arts & Culture
4,Yes,4,4,Apple,7,Yes,5,3,3,4,...,3,4,4,4,Yes,Male,5,7,Employed,Hospitality


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   trust_apple          133 non-null    object
 1   interest_computers   133 non-null    int64 
 2   age_computer         133 non-null    int64 
 3   user_pcmac           133 non-null    object
 4   appleproducts_count  133 non-null    int64 
 5   familiarity_m1       133 non-null    object
 6   f_batterylife        133 non-null    int64 
 7   f_price              133 non-null    int64 
 8   f_size               133 non-null    int64 
 9   f_multitasking       133 non-null    int64 
 10  f_noise              133 non-null    int64 
 11  f_performance        133 non-null    int64 
 12  f_neural             133 non-null    int64 
 13  f_synergy            133 non-null    int64 
 14  f_performanceloss    133 non-null    int64 
 15  m1_consideration     133 non-null    int64 
 16  m1_purch

In [17]:
from sklearn.preprocessing import LabelEncoder

X = df.drop('m1_purchase', axis=1)
y = (df['m1_purchase'] == "Yes").astype(int)

object_columns = X.select_dtypes(include=['object']).columns


label_encoder = LabelEncoder()

for col in object_columns:
    X[col] = label_encoder.fit_transform(X[col])

array([1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0])

#Decision tree sklearn

In [26]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix



# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Decision Tree classifier
clf = DecisionTreeClassifier()

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)



Accuracy: 0.6296296296296297
Confusion Matrix:
[[ 4  4]
 [ 6 13]]


# Decision tree from scratch


In [27]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature        # Feature index for the decision
        self.threshold = threshold    # Threshold value for the decision
        self.left = left              # Left subtree
        self.right = right            # Right subtree
        self.value = value            # Class label for leaf nodes

def gini_impurity(y):
    # Calculate Gini impurity for a set of labels
    total_samples = len(y)
    class_counts = {label: sum(y == label) for label in set(y)}
    impurity = 1.0 - sum((count / total_samples) ** 2 for count in class_counts.values())
    return impurity

def find_best_split(X, y):
    # Find the best split for a node using Gini impurity
    m, n = X.shape
    best_gini = float('inf')
    best_feature = None
    best_threshold = None

    for feature in range(n):
        unique_values = set(X[:, feature])
        for value in unique_values:
            left_mask = X[:, feature] <= value
            right_mask = ~left_mask

            left_gini = gini_impurity(y[left_mask])
            right_gini = gini_impurity(y[right_mask])

            weighted_gini = (sum(left_mask) / m) * left_gini + (sum(right_mask) / m) * right_gini

            if weighted_gini < best_gini:
                best_gini = weighted_gini
                best_feature = feature
                best_threshold = value

    return best_feature, best_threshold

def build_tree(X, y, max_depth=None):
    # Recursively build the decision tree
    if max_depth == 0 or len(set(y)) == 1:
        # Leaf node
        return Node(value=max(set(y), key=list(y).count))

    feature, threshold = find_best_split(X, y)
    left_mask = X[:, feature] <= threshold
    right_mask = ~left_mask

    left_subtree = build_tree(X[left_mask], y[left_mask], max_depth - 1 if max_depth else None)
    right_subtree = build_tree(X[right_mask], y[right_mask], max_depth - 1 if max_depth else None)

    return Node(feature=feature, threshold=threshold, left=left_subtree, right=right_subtree)

def predict_tree(node, sample):
    # Predict the class label for a sample using the decision tree
    if node.value is not None:
        # Leaf node
        return node.value
    if sample[node.feature] <= node.threshold:
        return predict_tree(node.left, sample)
    else:
        return predict_tree(node.right, sample)


In [28]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X_train


array([[ 1,  4,  5, ...,  6,  2,  9],
       [ 1,  5,  1, ...,  3,  3, 15],
       [ 1,  5,  5, ...,  2,  3, 16],
       ...,
       [ 1,  2,  9, ...,  3,  0,  7],
       [ 1,  3,  6, ...,  7,  2,  2],
       [ 1,  5,  1, ...,  1,  3, 15]])

In [29]:
tree = build_tree(X_train, y_train, max_depth=3)

# Make predictions
y_pred_own = [predict_tree(tree, sample) for sample in X_test]

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_own)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7037037037037037
